In [1]:
import pandas as pd
import numpy as np
from GPyOpt.methods.bayesian_optimization import BayesianOptimization
from modeltest import VAE

In [2]:
pairs_dev_train_path = '../pairsDevTrain.txt'
pairs_dev_test_path = '../pairsDevTest.txt'

def extract_pairs_for_development(pairs_dev_filepath: str):
    pairs = []
    with open(pairs_dev_filepath, 'r') as f:
        size = int(next(f))
        for i in range(size):
            line = next(f)
            name, num1, num2 = line.strip().split()
            pairs.append(((name, int(num1)), (name, int(num2))))
        for i in range(size):
            line = next(f)
            name1, num1, name2, num2 = line.strip().split()
            pairs.append(((name1, int(num1)), (name2, int(num2))))
    return pairs

pairs_dev_train = extract_pairs_for_development(pairs_dev_train_path)
pairs_dev_test = extract_pairs_for_development(pairs_dev_test_path)

In [3]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

dataset_path = '../lfw/'

class PairDataGenerator(Dataset):
    def __init__(self, pairs, transform=None):
        self.pairs = pairs

        composition = []
        if transform:
            composition.append(transform)
        composition.append(transforms.ToTensor())

        self.transform = transforms.Compose(composition)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, index):
        (name1, num1), (name2, num2) = self.pairs[index]
        # Assuming images are in a folder named 'lfw-deepfunneled'
        image_path1 = f'{dataset_path}/{name1}/{name1}_{num1:0>4}.jpg'
        image_path2 = f'{dataset_path}/{name2}/{name2}_{num2:0>4}.jpg'

        image1 = Image.open(image_path1).convert("RGB")
        image2 = Image.open(image_path2).convert("RGB")

        image1 = self.transform(image1)
        image2 = self.transform(image2)

        y_true = 1 if name1 == name2 else 0
        return image1, image2, y_true

In [4]:
train_pair_gen = PairDataGenerator(pairs_dev_train, transform=transforms.Resize((64, 64)))
test_pair_gen = PairDataGenerator(pairs_dev_test, transform=transforms.Resize((64, 64)))
train_pair_loader = DataLoader(train_pair_gen, batch_size=32)
test_pair_loader = DataLoader(test_pair_gen, batch_size=32)

In [5]:
vae = VAE((3, 64, 64), 1024, 3).cuda()
vae.load_state_dict(torch.load('trained/vae-(64, 64)-1024-3-1.dat')['state_dict'])
vae.eval()

VAE(
  (encoder_conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
  )
  (mu): Sequential(
    (0): Linear(in_features=8192, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Dropout(p=0.2, inplace=False)
  )
  (log_var): Sequential(
    (0): Linear(in_features=8192, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Dropout(p=0.2, inplace=False)
  )
  (decode

In [6]:
def to_numpy(pair_loader):
    left_latent_list = []
    right_latent_list = []
    value_list = []
    for left_images, right_images, value_batch in pair_loader:
        left_images = left_images.cuda()
        right_images = right_images.cuda()
        left_images_latent = vae.forward_encoder(left_images)[0].detach().cpu().numpy()
        right_images_latent = vae.forward_encoder(right_images)[0].detach().cpu().numpy()
        values = value_batch.detach().cpu().numpy()
        left_latent_list.append(left_images_latent)
        right_latent_list.append(right_images_latent)
        value_list.append(values)
    left_latent = np.concatenate(left_latent_list, axis=0)
    right_latent = np.concatenate(right_latent_list, axis=0)
    values = np.concatenate(value_list, axis=0)
    return left_latent, right_latent, values
train_left_latent, train_right_latent, train_values = to_numpy(train_pair_loader)
test_left_latent, test_right_latent, test_values = to_numpy(test_pair_loader)

In [7]:
dif = train_left_latent - train_right_latent
sq_dif = dif ** 2
sq_dif_sum = np.sum(sq_dif, axis=1)
train_dist_array = np.sqrt(sq_dif_sum)
train_min = train_dist_array.min()
train_max = train_dist_array.max()
print(train_min)
print(train_max)
print(len(train_dist_array))
train_dist_array

0.37028483
7.917598
2200


array([2.1989903, 2.8065174, 6.5347266, ..., 3.9846537, 5.6925116,
       2.3503323], dtype=float32)

In [8]:
dif = test_left_latent - test_right_latent
sq_dif = dif ** 2
sq_dif_sum = np.sum(sq_dif, axis=1)
test_dist_array = np.sqrt(sq_dif_sum)
test_min = test_dist_array.min()
test_max = test_dist_array.max()
print(test_min)
print(test_max)
print(len(test_dist_array))
test_dist_array

0.54111874
8.386483
1000


array([4.7947335 , 4.2095976 , 5.4025373 , 2.9110787 , 3.6225545 ,
       4.895653  , 4.3616405 , 4.538305  , 4.7886386 , 2.6920142 ,
       2.84272   , 2.4910893 , 3.0540814 , 3.0167825 , 4.2268686 ,
       3.1297805 , 1.7771438 , 5.4036984 , 1.8538308 , 1.6215715 ,
       2.9578512 , 2.8385153 , 1.9662426 , 2.7440317 , 3.3740404 ,
       6.2525024 , 2.6180542 , 3.595617  , 1.7258087 , 3.94167   ,
       0.78234226, 3.0225945 , 2.575028  , 3.3039777 , 2.2792032 ,
       4.5973306 , 3.62983   , 2.0513132 , 2.7243795 , 5.2130046 ,
       0.9514568 , 4.0553017 , 5.951153  , 2.7503443 , 4.6999593 ,
       5.6924505 , 3.0490599 , 3.884896  , 4.682737  , 4.19906   ,
       2.525636  , 4.823548  , 4.6054544 , 6.2674346 , 4.3438134 ,
       2.8449538 , 4.582164  , 2.9748445 , 2.6218185 , 4.233874  ,
       5.2937074 , 6.223753  , 5.156156  , 2.8767154 , 2.563845  ,
       5.5058002 , 2.558587  , 3.2211633 , 3.543159  , 2.8690448 ,
       3.3706844 , 2.1902614 , 3.2743616 , 4.244734  , 4.67207

In [9]:
def obj_func(threshold: float) -> float:
    decision = (train_dist_array < threshold).astype(int)
    accuracy = np.mean(decision == train_values)
    return -accuracy
def evaluate(dist_array: np.array, true: np.array, threshold: float) -> float:
    decision = (dist_array < threshold).astype(int)
    return np.mean(decision == true)

In [10]:
domain = [{'name': 'threshold', 'type': 'continuous', 'domain': (train_min, train_max)}]
max_iter = 50
BO = BayesianOptimization(f = obj_func, domain = domain)
BO.run_optimization(max_iter=max_iter)

In [11]:
opt_values = BO.x_opt
print(opt_values)

[3.79712657]


In [12]:
obj_func(*opt_values)

-0.5595454545454546

In [13]:
evaluate(test_dist_array, test_values, threshold=opt_values[0])

0.561

In [14]:
vae.conv_out_shape

torch.Size([1, 128, 8, 8])